In [ ]:
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

print(os.getcwd())


tableS3 = pd.read_csv(os.path.join(os.getcwd(), '..', 'data/NCB_ProteomeGuidedDiscovery_TableS3_260606a.csv'))

cluster_mappings = {1 : 'AR-VHL-Other',
               2: 'AR-T6N-Indole',
               3: 'AR-Other-Other',
               4: 'Txn-VHL/6N-Other',
               5: 'AR-VHL-Indole',
               6: 'AR-T5N-Other',
               7: 'Txn-Other-Other',
               8: 'AR-T6N/VHL-Pip',
               9: 'AR-T6N-Other',
               10: 'AR-T5N-Pip',
               11: 'AR-DHU-Pip',
               12: 'AR-Other-Pip',
               13: 'Non PROTAC',
               14: 'AR-L5N-Other',
               15: 'AR-L5N-Pip'} 


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))  # 2 rows, 2 cols

# KDE Plot: Toxic Probability by Cluster
sns.kdeplot(data=tableS3, x='Cluster', y='Toxic Probability', common_norm=False, ax=axes[0, 0])
axes[0, 0].set_title('KDE: Toxic Probability by Cluster')
axes[0, 0].set_xlabel('Chemical Series')

# Violin Plot: Toxic Probability by Cluster
sns.violinplot(data=tableS3, x='Cluster', y='Toxic Probability', hue='Cluster', common_norm=False,
               palette = 'pastel', ax=axes[0, 1])
axes[0, 1].set_title('Violin: Toxic Probability by Cluster')
axes[0, 1].set_xlabel('Chemical Series')
axes[0, 1].legend_.remove()


# KDE Plot: NDUFA5 by Cluster
sns.kdeplot(data=tableS3, x='Cluster', y='NDUFA5', common_norm=False, ax=axes[1, 0])
axes[1, 0].set_title('KDE: NDUFA5 by Cluster')
axes[1, 0].set_xlabel('Chemical Series')
axes[1, 0].set_ylabel('NDUFA5 Differential Expression')

toplot_violin = tableS3.copy()
toplot_violin['Chemistry'] = toplot_violin['Cluster'].map(cluster_mappings)

print(toplot_violin)

# Violin Plot: NDUFA5 by Cluster
sns.boxplot(data=toplot_violin, x='Chemistry', y='NDUFA5', hue='Chemistry', showfliers = False, 
            palette = 'pastel',  ax=axes[1, 1])
axes[1, 1].set_title('Violin: NDUFA5 by Cluster')
axes[1, 1].set_xlabel('Chemical Series')
axes[1, 1].set_ylabel('NDUFA5 Differential Expression')
for tick in axes[1, 1].get_xticklabels():
    tick.set_rotation(90)

plt.tight_layout()
plt.show()

In [ ]:
# KDE Plot: Toxic Probability by Cluster
sns.kdeplot(data=tableS3, x='Cluster', y='Toxic Probability', common_norm=False)
plt.title('KDE: Toxic Probability by Cluster')
plt.xlabel('Chemical Series')
plt.show()

In [ ]:


cluster_summary = tableS3.groupby('Cluster')['Toxic Probability'].describe().sort_values(
    ['mean', 'std'], ascending=False)

cluster_summary['chemistry'] = cluster_summary.index.map(cluster_mappings)

cluster_summary



### 🧹 Data Cleaning and Filtering Strategy

This analysis applied data filtering process to respect the following profiles:

1. **Tox Score Filtering**  
   Only chemical series with a **median safety prediction score ≥ 66%** were retained.  
   This threshold is based on **troglitazone**, a drug discontinued due to hepatotoxicity, which had a median score of 66%.  
   Any series scoring **lower than troglitazone** was excluded to focus on toxic profiles.

2. **Minimum Sample Size Theshold**  
   To ensure statistical robustness, only clusters with **n > 5 compounds** were included in the final set.

3. **Bimodal Series Selection**  
   From the filtered set, only **clusters exhibiting bimodal distributions** in their toxicity probability were selected.  
   This step aims to capture series with distinct sub-populations of low and high predicted toxicity.

4. **Variability Emphasis**  
   Among eligible clusters, we prioritized those with **higher standard deviation** in toxicity predictions,  
   indicating a greater spread and potential diversity in toxicity behavior across compounds within a series.

The result is a curated selection of chemically meaningful and statistically robust clusters, enriched for variability and toxicity diversity.

In [ ]:
#troglitazone safety thresh (toxicity > 50)
cluster_filter1_idx = cluster_summary.loc[cluster_summary['mean']>0.50].index

cluster_filter1 = cluster_summary.loc[cluster_filter1_idx]

#minimum sample size thresh (n > 30)
n_thresh = 5
cluster_filter2_idx = cluster_filter1.loc[cluster_filter1['count']>n_thresh].index

cluster_filter2 = cluster_filter1.loc[cluster_filter2_idx]

toplot = tableS3.loc[tableS3['Cluster'].isin(cluster_filter2_idx)].copy()
toplot['Chemistry'] = toplot['Cluster'].map(cluster_mappings)

# bimodality and variability check ## DROP OUT CLUSTER 6 (Not Bimodal) & 8 FOR PLOTTING (Other is mixed N)

toplot_bimodal = toplot.loc[toplot['Cluster'].isin([1,10,11,14,15])]
sns.violinplot(toplot_bimodal, x = 'Chemistry', y = 'Toxic Probability',
               hue = 'Chemistry', palette = 'pastel', common_norm=False)
plt.legend().remove()
plt.xticks(rotation = 90)
plt.xlabel(None)
plt.show()


In [ ]:
export_columns = ['Unnamed: 0', 'Toxic Probability', 'Drug', 'Cluster','Chemistry', 'NDUFA5']

export_df = toplot[toplot['Chemistry']!='AR-Other-Pip']

toxic_subset = export_df.loc[export_df['Toxic Probability']>0.66].loc[export_df['NDUFA5']>0.44][export_columns]

toxic_subset.set_index('Unnamed: 0', inplace = True)

toxic_subset.to_csv(os.path.join(os.getcwd(), '..', 'data/HTMSdrugsafety_Toxic_Subset_250522a.csv'))

nontoxic_subset = export_df.loc[export_df['Toxic Probability']<0.35].loc[export_df['NDUFA5']<0.01][export_columns]

nontoxic_subset.set_index('Unnamed: 0', inplace = True)

#nontoxic_subset.to_csv(os.path.join(os.getcwd(), '..', 'data/HTMSdrugsafety_Safe_SuperStrict_250522a.csv'))
nontoxic_subset.to_csv(os.path.join(os.getcwd(), '..', 'data/HTMSdrugsafety_Safe_Strict_250522a.csv'))

print(nontoxic_subset.shape)
nontoxic_subset

In [ ]:
print(toxic_subset.shape)

toxic_subset

In [ ]:
VHLsubset = tableS3.loc[tableS3['Cluster']==1]

VHLexport = VHLsubset[['Unnamed: 0', 'Toxic Probability', 'Drug', 'Cluster','NDUFA5']]

VHLexport.set_index('Unnamed: 0', inplace = True)

VHLexport = VHLexport.assign(Chemistry='AR-VHL-Other').loc[VHLexport['NDUFA5']<0.1]

VHLexport

#VHLexport.to_csv(os.path.join(os.getcwd(), '..', 'data/HTMSdrugsafety_VHLsafe_250522a.csv'))






In [ ]:
toxic_subset['Pred'] = 'Toxic'
nontoxic_subset['Pred'] = 'Safe'
VHLexport['Pred'] = 'Safe'

total_subset = pd.concat([toxic_subset, nontoxic_subset, VHLexport])

sns.barplot(total_subset.loc[total_subset['Cluster'].isin([1,6,11,14,15])], x = 'Chemistry', y = 'Toxic Probability', hue = 'Pred')
plt.xticks(rotation = 90)
plt.ylim([-0.1,1])

In [ ]:
degrader_data = pd.read_excel(os.path.join(os.getcwd(), '..', 'data/Figure4_ARdegdata_SafetyScores_250523a.xlsx'),
                              names=['Compound', 'Unnamed: 0', 'AR_DEG_IC50', 'Label'])

degrader_data.iloc[:,1] = degrader_data.iloc[:,1].str.replace('-','_')

total_data = degrader_data.merge(total_subset, on='Unnamed: 0', how = 'left')

select_data = total_data.loc[total_data['Label'].isin(degrader_data['Label'].value_counts().index)]

select_data = total_data.loc[~total_data['Label'].isin(['Thalidomide Degrader', 'Lenalidomide Degrader'])]

plt.figure(figsize=(5, 5))  # width=8, height=6 inches
sns.scatterplot(select_data, x = 'AR_DEG_IC50', y = 'Toxic Probability', hue = 'Label', s = 400)
plt.savefig(os.path.join(os.getcwd(), '..', 'figures/Figure_5_ARDEG_Scatter.pdf'))
